# Decorators (Memoization)
### No, it is not a typo.
Decorators can do advanced functionality, like caching.

Lets define a simple function

In [5]:
def fib(target: int) -> int:
    """This function generates a Fibonacci sequence item, specified by `target`"""
    if target == 0:
        return 0
    elif target <= 2:
        return 1
    else:
        print("calculationg Fib for {0}".format(target))
        return fib(target - 1) + fib(target - 2)

fib(8)


calculationg Fib for 8
calculationg Fib for 7
calculationg Fib for 6
calculationg Fib for 5
calculationg Fib for 4
calculationg Fib for 3
calculationg Fib for 3
calculationg Fib for 4
calculationg Fib for 3
calculationg Fib for 5
calculationg Fib for 4
calculationg Fib for 3
calculationg Fib for 3
calculationg Fib for 6
calculationg Fib for 5
calculationg Fib for 4
calculationg Fib for 3
calculationg Fib for 3
calculationg Fib for 4
calculationg Fib for 3


21

What a mess, this recursion did alot of repeptitve work. Just look how many times we calculated Fib of number 3! With large numbers this will quickly accumulate!

Lets add a memoization wrapper to it.

In [10]:
def memory(fn):
    from functools import wraps
    dict1 = dict()
    @wraps(fn)
    def inner(target):
        if target in dict1:
            print("Retrieved Fib of {0}".format(target))
        else:
            dict1[target] = fn(target)
            print("   Stored Fib of {0}".format(target))
        return dict1[target]
    return inner

@memory
def fib(target: int) -> int:
    """This function generates a Fibonacci sequence item, specified by `target`"""
    if target == 0:
        return 0
    elif target <= 2:
        return 1
    else:
        print("Calculating Fib for {0}".format(target))
        return fib(target - 1) + fib(target - 2)
    
fib(10)

Calculating Fib for 10
Calculating Fib for 9
Calculating Fib for 8
Calculating Fib for 7
Calculating Fib for 6
Calculating Fib for 5
Calculating Fib for 4
Calculating Fib for 3
   Stored Fib of 2
   Stored Fib of 1
   Stored Fib of 3
Retrieved Fib of 2
   Stored Fib of 4
Retrieved Fib of 3
   Stored Fib of 5
Retrieved Fib of 4
   Stored Fib of 6
Retrieved Fib of 5
   Stored Fib of 7
Retrieved Fib of 6
   Stored Fib of 8
Retrieved Fib of 7
   Stored Fib of 9
Retrieved Fib of 8
   Stored Fib of 10


55

Much better, just a fraction of calculations.

Is the metadata preserved?

In [12]:
help(fib)

Help on function fib in module __main__:

fib(target: int) -> int
    This function generates a Fibonacci sequence item, specified by `target`



Sure is!

What if we want to pass an argument to the decorator, like with `wraps`? Well, we'll have to make a...
# Decorator Factory
Factory is made with 2 nested functions: factory(decorator(function(call)))

In [17]:
def memory(max_items: int = 8):
    """This is a memory decorator factory"""
    def memory1(fn):
        """This is a memory decorator generator"""
        from functools import wraps
        dict1 = dict()
        @wraps(fn)
        def inner(target: int):
            """This is a memory decorating function"""
            if target in dict1:
                print("Retrieving value for {0}".format(target))
                value = dict1[target]
            else:
                value = fn(target)
                if len(dict1) >= max_items:
                    print("Unable to store value for {0}, memory is full".format(target))
                else:
                    print("Storing value for {0}".format(target))
                    dict1[target] = value
            return value
        return inner
    return memory1

@memory(6)
def fib(target: int) -> int:
    """This function generates a Fibonacci sequence item, specified by `target`"""
    if target == 0:
        return 0
    elif target <= 2:
        return 1
    else:
        print("Calculating Fib for {0}".format(target))
        return fib(target - 1) + fib(target - 2)
    
fib(10)
help(fib)

Calculating Fib for 10
Calculating Fib for 9
Calculating Fib for 8
Calculating Fib for 7
Calculating Fib for 6
Calculating Fib for 5
Calculating Fib for 4
Calculating Fib for 3
Storing value for 2
Storing value for 1
Storing value for 3
Retrieving value for 2
Storing value for 4
Retrieving value for 3
Storing value for 5
Retrieving value for 4
Storing value for 6
Retrieving value for 5
Unable to store value for 7, memory is full
Retrieving value for 6
Unable to store value for 8, memory is full
Calculating Fib for 7
Retrieving value for 6
Retrieving value for 5
Unable to store value for 7, memory is full
Unable to store value for 9, memory is full
Calculating Fib for 8
Calculating Fib for 7
Retrieving value for 6
Retrieving value for 5
Unable to store value for 7, memory is full
Retrieving value for 6
Unable to store value for 8, memory is full
Unable to store value for 10, memory is full
Help on function fib in module __main__:

fib(target: int) -> int
    This function generates a Fi

We can see that larger items had to be regenerated because the given function memory limit was reached. It was specified via decorator factory parameter. This function call is nested, so if we would call it in a traditional way, it would look like this:

In [19]:
def fib(target: int) -> int:
    """This function generates a Fibonacci sequence item, specified by `target`"""
    if target == 0:
        return 0
    elif target <= 2:
        return 1
    else:
        print("Calculating Fib for {0}".format(target))
        return fib(target - 1) + fib(target - 2)

fib = memory(6)(fib) # o_O

fib(10)
help(fib)

Calculating Fib for 10
Calculating Fib for 9
Calculating Fib for 8
Calculating Fib for 7
Calculating Fib for 6
Calculating Fib for 5
Calculating Fib for 4
Calculating Fib for 3
Storing value for 2
Storing value for 1
Storing value for 3
Retrieving value for 2
Storing value for 4
Retrieving value for 3
Storing value for 5
Retrieving value for 4
Storing value for 6
Retrieving value for 5
Unable to store value for 7, memory is full
Retrieving value for 6
Unable to store value for 8, memory is full
Calculating Fib for 7
Retrieving value for 6
Retrieving value for 5
Unable to store value for 7, memory is full
Unable to store value for 9, memory is full
Calculating Fib for 8
Calculating Fib for 7
Retrieving value for 6
Retrieving value for 5
Unable to store value for 7, memory is full
Retrieving value for 6
Unable to store value for 8, memory is full
Unable to store value for 10, memory is full
Help on function fib in module __main__:

fib(target: int) -> int
    This function generates a Fi

This weird syntax works. It's weird like that because functions are objects.